This notebook is associated with Section 6 and Appendix B of the paper ["A fundamental non-classical logic"](https://arxiv.org/abs/2207.06993) by [Wesley H. Holliday](mailto:wesholliday@berkeley.edu).

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s [interface](https://www.nltk.org/howto/inference.html) to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to verify basic facts about the "preconditionals" and "preimplications" discussed in the paper.

To view the notebook online, type the URL of this notebook (https://github.com/wesholliday/fundamental-logic/blob/main/implication-axioms.ipynb) into the location field at https://nbviewer.org. GitHub's preview of the notebook might not show all the output that was generated.

## Outline

**1. [Lattice axioms](#1)**

**2. [Bounded lattice axioms](#2)**

**3. [Preconditionals](#3)**

**4. [Preimplications](#4)**

**5. [Relating preconditionals and preimplications](#5)**

In [1]:
from nltk.test.inference_fixt import setup_module 

setup_module()

from nltk import *
from nltk.sem.drt import DrtParser
from nltk.sem import logic
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

## 1. Lattice axioms<a id='1'></a>

In [2]:
or_id = read_expr('Or(x,x) = x')
and_id = read_expr('And(x,x) = x')

or_comm = read_expr('Or(x,y) = Or(y,x)')
and_comm = read_expr('And(x,y) = And(y,x)')

or_assoc = read_expr('Or(x,Or(y,z)) = Or(Or(x,y),z)')
and_assoc = read_expr('And(x,And(y,z)) = And(And(x,y),z)')

or_absorp = read_expr('Or(x,And(x,y)) = x')
and_absorp = read_expr('And(x,Or(x,y)) = x')

lattice = [or_id, and_id, or_comm, and_comm, or_assoc, and_assoc, or_absorp, and_absorp]

In [3]:
#By including a definition of the covering relation,
#one can quickly read off Hasse diagrams of lattices from the Mace4 output.
cover_def = read_expr('covered_by(x,y) <-> (-(x=y) & x = And(x,y) & -exists z.(-(z=x) & -(z=y) & x = And(x,z) & z = And(z,y)))')

## 2. Bounded lattice axioms<a id='2'></a>

In [4]:
bot = read_expr('Or(x,Bot) = x')
top = read_expr('And(x,Top) = x')

bounded_lattice = lattice + [bot,top]

# 3. Preconditionals

In [5]:
precon1 = read_expr('Cond(Top,x) = And(Cond(Top,x),x)')
precon2 = read_expr('And(x,y) = And(And(x,y),Cond(x,y))')
precon3 = read_expr('Cond(x,y) = And(Cond(x,y),Cond(x,And(x,y)))')
precon4 = read_expr('x = And(x,z) -> Cond(z,Cond(x,y)) = And(Cond(z,Cond(x,y)), Cond(x,y))')
precon5 = read_expr('x = And(x,y) -> Cond(z,x) = And(Cond(z,x), Cond(z,y))')

precon = [precon1,precon2,precon3,precon4,precon5]

In [6]:
#Verify that the preconditional axioms are independent

for ax in precon:
    print(f"Independence of {ax}: \n")
    goal = ax
    mb = MaceCommand(goal, assumptions = bounded_lattice + [axiom for axiom in precon if not axiom == ax])
    mb.build_model()
    print(mb.model(format='cooked'))

Independence of (Cond(Top,x) = And(Cond(Top,x),x)): 

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 0.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

Cond(0,0) = 1.
Cond(0,1) = 1.
Cond(1,0) = 1.
Cond(1,1) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.

Independence of (And(x,y) = And(And(x,y),Cond(x,y))): 

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 1.

c2 = 1.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

Cond(0,0) = 0.
Cond(0,1) = 0.
Cond(1,0) = 0.
Cond(1,1) = 0.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.

Independence of (Cond(x,y) = And(Cond(x,y),Cond(x,And(x,y)))): 

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 0.

c2 = 1.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

Cond(0,0) = 0.
Cond(0,1) = 1.
Cond(1,0) = 0.
Cond(1,1) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.

Independence of ((x = And(x,z)) -> (Con

In [7]:
#Verify that every bounded lattice can be equipped with a preconditional using the following definition

precon_def = read_expr('(x=And(x,y) -> Cond(x,y)=Top) & (-(x=And(x,y)) -> Cond(x,y)=And(x,y))')

for ax in precon:
    print("")
    print(f"Proof of {ax}:")
    print("")
    goal = ax
    prover = Prover9Command(goal, assumptions = bounded_lattice + [precon_def])
    prover.prove()
    print(prover.proof())


Proof of (Cond(Top,x) = And(Cond(Top,x),x)):

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 8768 was started by halcrow on iMac.local,
Sat Oct 29 17:53:55 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 14.
% Level of proof is 5.
% Maximum clause weight is 12.000.
% Given clauses 33.


1 (x = And(x,y) -> Cond(x,y) = Top) & (x != And(x,y) -> Cond(x,y) = And(x,y)).  [assumption].
2 Cond(Top,x) = And(Cond(Top,x),x).  [goal].
4 And(x,x) = x.  [assumption].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
15 And(x,y) != x | Cond(x,y) = Top.  [clausify(1)].
16 And(x,y

In [8]:
#Verify that every bounded lattice with a precomplementation gives rise to a preconditional as follows

#Warning: this may take a long time

antitone = read_expr('x=And(x,y) -> Not(y)=And(Not(x),Not(y))')
top_to_bot = read_expr('Not(Top)=Bot')

precon_from_precomp_def = read_expr('Cond(x,y)=Or(Not(x),And(x,y))')

for ax in precon:
    print("")
    print(f"Proof of {ax}:")
    print("")
    goal = ax
    prover = Prover9Command(goal, assumptions = bounded_lattice + [antitone,top_to_bot,precon_from_precomp_def],timeout=100000)
    prover.prove()
    print(prover.proof())


Proof of (Cond(Top,x) = And(Cond(Top,x),x)):

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 8778 was started by halcrow on iMac.local,
Sat Oct 29 17:53:56 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 14.
% Level of proof is 3.
% Maximum clause weight is 13.000.
% Given clauses 9.


2 Cond(Top,x) = And(Cond(Top,x),x).  [goal].
4 And(x,x) = x.  [assumption].
5 Or(x,y) = Or(y,x).  [assumption].
6 And(x,y) = And(y,x).  [assumption].
9 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
10 And(And(x,y),z) = And(x,And(y,z)).  [copy(9),flip(a)].
13 Or(x,Bot) = x.  [assumption].
17 Not(Top) 

# 4. Preimplications

In [9]:
preimp1_left_to_right = read_expr('x = And(x,Imp(Top,x))')
preimp1_right_to_left = read_expr('Imp(Top,x) = And(Imp(Top,x),x)')
preimp2 = read_expr('Imp(x,Imp(x,y)) = And(Imp(x,Imp(x,y)), Imp(x,y))')
preimp3 = read_expr('x = And(x,y) -> Imp(y,z) = And(Imp(y,z), Imp(x,z))')
preimp4 = read_expr('x = And(x,y) -> Imp(z,x) = And(Imp(z,x), Imp(z,y))')

preimp = [preimp1_left_to_right, preimp1_right_to_left, preimp2, preimp3, preimp4]

In [10]:
#Verify that the preimplication axioms are independent

for ax in preimp:
    print(f"Independence of {ax}: \n")
    goal = ax
    mb = MaceCommand(goal, assumptions = bounded_lattice + [axiom for axiom in preimp if not axiom == ax])
    mb.build_model()
    print(mb.model(format='cooked'))

Independence of (x = And(x,Imp(Top,x))): 

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 1.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

Imp(0,0) = 0.
Imp(0,1) = 0.
Imp(1,0) = 0.
Imp(1,1) = 0.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.

Independence of (Imp(Top,x) = And(Imp(Top,x),x)): 

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 0.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

Imp(0,0) = 1.
Imp(0,1) = 1.
Imp(1,0) = 1.
Imp(1,1) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.

Independence of (Imp(x,Imp(x,y)) = And(Imp(x,Imp(x,y)),Imp(x,y))): 

% number = 1
% seconds = 0

% Interpretation of size 3

Bot = 0.

Top = 1.

c1 = 0.

c2 = 0.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.

Imp(0,0) = 2.
Imp(0,1) = 1.
Imp(0,2) = 1.
Imp(1,0) = 0.
Imp(1,1) = 1.
Imp(1,2) = 2.
Imp(2,0) = 0.
Imp(2,1) = 1.

In [11]:
#Verify that every bounded lattice can be equipped with a preimplication using the following definition

preimp_def = read_expr('(x=And(x,y) -> Imp(x,y)=Top) & (-(x=And(x,y)) -> Imp(x,y)=y)')

for ax in preimp:
    print("")
    print(f"Proof of {ax}:")
    print("")
    goal = ax
    prover = Prover9Command(goal, assumptions = bounded_lattice + [preimp_def])
    prover.prove()
    print(prover.proof())


Proof of (x = And(x,Imp(Top,x))):

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 33474 was started by halcrow on iMac.local,
Sun Oct 30 05:16:41 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 12.
% Level of proof is 3.
% Maximum clause weight is 10.000.
% Given clauses 13.


1 (x = And(x,y) -> Imp(x,y) = Top) & (x != And(x,y) -> Imp(x,y) = y).  [assumption].
2 x = And(x,Imp(Top,x)).  [goal].
4 And(x,x) = x.  [assumption].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
15 And(x,y) != x | Imp(x,y) = Top.  [clausify(1)].
16 And(x,y) = x | Imp(x,y) = y.  [clausif

In [12]:
#Verify that given an orthocomplementation Not, the implication Imp(x,y) defined by Not(x and Not(y)) is a preimplication.

antitone = read_expr('x=And(x,y) -> Not(y)=And(Not(x),Not(y))')
semicomp = read_expr('And(x,Not(x)) = Bot')
DNI = read_expr('x=And(x,Not(Not(x)))')
DNE = read_expr('Not(Not(x))=And(Not(Not(x)),x)')
LEM = read_expr('Or(x,Not(x)) = Top')
orthocomp = [antitone, semicomp, DNI, DNE, LEM]

ortho_preimp_def = read_expr('Imp(x,y) = Not(And(x,Not(y)))')

for ax in preimp:
    print("")
    print(f"Proof of {ax}:")
    print("")
    goal = ax
    prover = Prover9Command(goal, assumptions = bounded_lattice + orthocomp + [ortho_preimp_def])
    prover.prove()
    print(prover.proof())


Proof of (x = And(x,Imp(Top,x))):

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 33526 was started by halcrow on iMac.local,
Sun Oct 30 05:17:41 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 13.
% Level of proof is 3.
% Maximum clause weight is 9.000.
% Given clauses 10.


2 x = And(x,Imp(Top,x)).  [goal].
4 And(x,x) = x.  [assumption].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
18 x = And(x,Not(Not(x))).  [assumption].
19 And(x,Not(Not(x))) = x.  [copy(18),flip(a)].
20 Not(Not(x)) = And(Not(Not(x)),x).  [assumption].
21 Not(Not(x)) = x.  [copy(20),rewri

In [13]:
#Verify that every bounded lattice with a precomplementation gives rise to a preimplication as follows

antitone = read_expr('x=And(x,y) -> Not(y)=And(Not(x),Not(y))')
top_to_bot = read_expr('Not(Top)=Bot')

preimp_from_precomp_def = read_expr('Imp(x,y)=Or(Not(x),y)')

for ax in preimp:
    print("")
    print(f"Proof of {ax}:")
    print("")
    goal = ax
    prover = Prover9Command(goal, assumptions = bounded_lattice + [antitone,top_to_bot,preimp_from_precomp_def])
    prover.prove()
    print(prover.proof())


Proof of (x = And(x,Imp(Top,x))):

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 33583 was started by halcrow on iMac.local,
Sun Oct 30 05:18:41 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 9.
% Level of proof is 3.
% Maximum clause weight is 8.000.
% Given clauses 8.


2 x = And(x,Imp(Top,x)).  [goal].
5 Or(x,y) = Or(y,x).  [assumption].
12 And(x,Or(x,y)) = x.  [assumption].
17 Not(Top) = Bot.  [assumption].
18 Imp(x,y) = Or(Not(x),y).  [assumption].
19 And(c1,Imp(Top,c1)) != c1.  [deny(2)].
20 And(c1,Or(Bot,c1)) != c1.  [copy(19),rewrite([18(4),17(3)])].
35 And(x,Or(y,x)) = x.  [

## Protoimplications

In [14]:
identity = read_expr('Imp(x,x)=Top')
modus_ponens = read_expr('And(x,Imp(x,y))= And(And(x,Imp(x,y)),y)')

protoimp = preimp + [identity,modus_ponens]

In [15]:
#Verify that the identity axiom does not already follow from preimplication axioms

goal = identity
mb = MaceCommand(goal, assumptions = bounded_lattice + preimp + [modus_ponens])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 0.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

Imp(0,0) = 0.
Imp(0,1) = 1.
Imp(1,0) = 0.
Imp(1,1) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.



In [16]:
#Verify that modus ponens does not already follow from preimplication axioms

goal = modus_ponens
mb = MaceCommand(goal, assumptions = bounded_lattice + preimp + [identity])
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 3

Bot = 0.

Top = 1.

c1 = 2.

c2 = 0.

And(0,0) = 0.
And(0,1) = 0.
And(0,2) = 0.
And(1,0) = 0.
And(1,1) = 1.
And(1,2) = 2.
And(2,0) = 0.
And(2,1) = 2.
And(2,2) = 2.

Imp(0,0) = 1.
Imp(0,1) = 1.
Imp(0,2) = 1.
Imp(1,0) = 0.
Imp(1,1) = 1.
Imp(1,2) = 2.
Imp(2,0) = 1.
Imp(2,1) = 1.
Imp(2,2) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(0,2) = 2.
Or(1,0) = 1.
Or(1,1) = 1.
Or(1,2) = 1.
Or(2,0) = 2.
Or(2,1) = 1.
Or(2,2) = 2.



## Ultraweak pseudoimplications

In [17]:
ultraweak_pseudo_ax = read_expr('x = And(x, Imp(Imp(x,y), y))')
ultraweak_pseudo_ax_equiv = read_expr('x=And(x,Imp(z,y)) -> z=And(z,Imp(x,y))')

ultraweak_pseudo = preimp + [ultraweak_pseudo_ax]

In [18]:
# Verify Lemma B.5 from 1 to 2

goal = ultraweak_pseudo_ax_equiv
prover = Prover9Command(goal, assumptions = bounded_lattice + preimp + [ultraweak_pseudo_ax])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 33816 was started by halcrow on iMac.local,
Sun Oct 30 05:23:41 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.13 (+ 0.01) seconds.
% Length of proof is 16.
% Level of proof is 5.
% Maximum clause weight is 16.000.
% Given clauses 223.


1 x = And(x,y) -> Imp(y,z) = And(Imp(y,z),Imp(x,z)).  [assumption].
3 x = And(x,Imp(y,z)) -> y = And(y,Imp(x,z)).  [goal].
7 And(x,y) = And(y,x).  [assumption].
10 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
11 And(And(x,y),z) = And(x,And(y,z)).  [copy(10),flip(a)].
22 And(x,y) != x | Imp(y,z) = And(Imp(y,z),Imp(x,z)).  [clausify(1)].
23 And

In [19]:
# Verify Lemma B.5 from 2 to 1

goal = ultraweak_pseudo_ax
prover = Prover9Command(goal, assumptions = bounded_lattice + preimp + [ultraweak_pseudo_ax_equiv])
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 33860 was started by halcrow on iMac.local,
Sun Oct 30 05:24:41 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 7.
% Level of proof is 3.
% Maximum clause weight is 14.000.
% Given clauses 15.


3 x = And(x,Imp(y,z)) -> y = And(y,Imp(x,z)).  [assumption].
4 x = And(x,Imp(Imp(x,y),y)).  [goal].
6 And(x,x) = x.  [assumption].
27 And(x,Imp(y,z)) != x | And(y,Imp(x,z)) = y.  [clausify(3)].
28 And(c1,Imp(Imp(c1,c2),c2)) != c1.  [deny(4)].
61 And(x,Imp(Imp(x,y),y)) = x.  [hyper(27,a,6,a)].
62 $F.  [resolve(61,a,28,a)].

==========================

In [20]:
#Verify that the ultraweak_pseudo axiom does not already follow from the preimplication axioms

goal = ultraweak_pseudo_ax
mb = MaceCommand(goal, assumptions = bounded_lattice + preimp)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 2

Bot = 0.

Top = 1.

c1 = 1.

c2 = 0.

And(0,0) = 0.
And(0,1) = 0.
And(1,0) = 0.
And(1,1) = 1.

Imp(0,0) = 0.
Imp(0,1) = 1.
Imp(1,0) = 0.
Imp(1,1) = 1.

Or(0,0) = 0.
Or(0,1) = 1.
Or(1,0) = 1.
Or(1,1) = 1.



## Weak pseudoimplications

In [21]:
weak_pseudoimp = protoimp + [ultraweak_pseudo_ax]

In [22]:
relative_pseudo_left_to_right = read_expr('And(x,y)=And(And(x,y),z) -> x=And(x,Imp(y,z))')

In [23]:
#Verify that the left-to-right direction of the relative pseudocomplementation principle 
#does not follow from the weak pseudoimplication axioms

goal = relative_pseudo_left_to_right
mb = MaceCommand(goal, assumptions = weak_pseudoimp)
mb.build_model()
print(mb.model(format='cooked'))

% number = 1
% seconds = 0

% Interpretation of size 3

Top = 0.

c1 = 0.

c2 = 1.

c3 = 2.

And(0,0) = 0.
And(0,1) = 2.
And(0,2) = 2.
And(1,0) = 1.
And(1,1) = 1.
And(1,2) = 2.
And(2,0) = 2.
And(2,1) = 2.
And(2,2) = 2.

Imp(0,0) = 0.
Imp(0,1) = 1.
Imp(0,2) = 2.
Imp(1,0) = 0.
Imp(1,1) = 0.
Imp(1,2) = 2.
Imp(2,0) = 0.
Imp(2,1) = 0.
Imp(2,2) = 0.



In [24]:
#Verify that the preimplication defined for any bounded lattice above is in fact a weak pseudoimplication

for ax in weak_pseudoimp:
    print("")
    print(f"Proof of {ax}:")
    print("")
    goal = ax
    prover = Prover9Command(goal, assumptions = bounded_lattice + [preimp_def])
    prover.prove()
    print(prover.proof())


Proof of (x = And(x,Imp(Top,x))):

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 34145 was started by halcrow on iMac.local,
Sun Oct 30 06:00:48 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 12.
% Level of proof is 3.
% Maximum clause weight is 10.000.
% Given clauses 13.


1 (x = And(x,y) -> Imp(x,y) = Top) & (x != And(x,y) -> Imp(x,y) = y).  [assumption].
2 x = And(x,Imp(Top,x)).  [goal].
4 And(x,x) = x.  [assumption].
6 And(x,y) = And(y,x).  [assumption].
14 And(x,Top) = x.  [assumption].
15 And(x,y) != x | Imp(x,y) = Top.  [clausify(1)].
16 And(x,y) = x | Imp(x,y) = y.  [clausif

# 5. Relating preconditionals and preimplications

In [25]:
#Verify that every bounded lattice equipped with a preimplication gives rise to a preconditional as follows

precon_def = read_expr('Cond(x,y) = Imp(x,And(x,y))')

for ax in precon:
    print("")
    print(f"Proof of {ax}:")
    print("")
    goal = ax
    prover = Prover9Command(goal, assumptions = bounded_lattice + preimp + [precon_def],timeout=10000)
    prover.prove()
    print(prover.proof())


Proof of (Cond(Top,x) = And(Cond(Top,x),x)):

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 34203 was started by halcrow on iMac.local,
Sun Oct 30 06:01:48 2022
The command was "/usr/local/bin/prover9/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 14.
% Level of proof is 4.
% Maximum clause weight is 11.000.
% Given clauses 6.


3 Cond(Top,x) = And(Cond(Top,x),x).  [goal].
5 And(x,x) = x.  [assumption].
7 And(x,y) = And(y,x).  [assumption].
10 And(x,And(y,z)) = And(And(x,y),z).  [assumption].
11 And(And(x,y),z) = And(x,And(y,z)).  [copy(10),flip(a)].
16 x = And(x,Imp(Top,x)).  [assumption].
17 And(x,Imp(Top,x)) = x.  [copy(16),